In [17]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [18]:
filename = "ABox.ttl"

In [19]:
text1 = '''CQ_5.1
Return the assets that are part of the `house` asset and the asset percentage they represent, in descending order.
'''

query1 = '''
PREFIX tbox: <http://purl.org/sirius/ontology/development/05/schema/>
PREFIX abox: <http://purl.org/sirius/ontology/development/05/data/>

SELECT ?asset ?asset_part ?percentage
WHERE {
  ?asset a tbox:HeritageAsset ;
        tbox:hasPart ?asset_part .
  ?asset_part tbox:representsAssetPercentage ?percentage .
}

'''

In [20]:
text2 = '''CQ_5.2
Return the assets that are part of the `house` asset and the contributing values assigned to them, along with their score, dimension, aspect, note, documentation, and time interval.
'''

query2 = '''
PREFIX tbox: <http://purl.org/sirius/ontology/development/05/schema/>
PREFIX abox: <http://purl.org/sirius/ontology/development/05/data/>

SELECT DISTINCT ?asset_part ?value ?score ?dimension ?aspect ?note ?document ?time_interval_start ?time_interval_end
WHERE {
    ?asset a tbox:HeritageAsset ;
            tbox:hasPart ?asset_part .
    ?value_assessment tbox:assignsValueTo ?asset_part ;
                        tbox:assignsValue ?value .
    ?value tbox:hasScore ?score ;
            tbox:withDimension ?dimension ;
            tbox:withinAspect ?aspect ;
            tbox:hasNote ?note ;
            tbox:isDocumentedBy ?document ;
            tbox:atTime ?time_interval .
    ?time_interval tbox:hasStartDate ?time_interval_start ;
                    tbox:hasEndDate ?time_interval_end .
}
'''

In [21]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_5.1
Return the assets that are part of the `house` asset and the asset percentage they represent, in descending order.

+-----------------------------------------------------------+---------------------------------------------------------------+--------------+
| asset                                                     | asset_part                                                    |   percentage |
|-----------------------------------------------------------+---------------------------------------------------------------+--------------|
| http://purl.org/sirius/ontology/development/05/data/house | http://purl.org/sirius/ontology/development/05/data/building  |           50 |
| http://purl.org/sirius/ontology/development/05/data/house | http://purl.org/sirius/ontology/development/05/data/furniture |           20 |
| http://purl.org/sirius/ontology/development/05/data/house | http://purl.org/sirius/ontology/development/05/data/mosaics   |           20 |
| http://purl.org/sirius/ontolo